# Data inspection - Check rendundacies and make the dataframe slimmer


In [123]:
import pandas as pd

In [124]:
df = pd.read_csv ('../data/accidents.csv', sep=';')

In [125]:
# First look at the dataframe
df.head(3)

,IdUsager,Date,PV,Arrondissement,Mode,Catégorie,Gravité,Age,Genre,Milieu,...,Blessés Légers,Blessés hospitalisés,Tué,Résumé,Coordonnées,Nom arrondissement,arronco,arrondgeo,Coordonnées.1,Nom arrondissement.1
0,2389401,2017-04-03,3527,75111,Piéton,Piéton,Blessé léger,62.0,Feminin,En-Agg,...,1.0,NaN,NaN,"Accident Léger non mortel, En agglomération, H...","48.855, 2.36867",Paris 4e Arrondissement,75104,"{""coordinates"": [[[[2.369123881, 48.853166231]...","48.855, 2.36867",Paris 4e Arrondissement
1,2388322,2017-08-28,9113,75108,2 Roues Motorisées,Conducteur,Blessé léger,30.0,Masculin,En-Agg,...,1.0,NaN,NaN,"Accident Léger non mortel, En agglomération, E...","48.8667, 2.3013",Paris 8e Arrondissement,75108,"{""coordinates"": [[[[2.301737288, 48.863496077]...","48.8667, 2.3013",Paris 8e Arrondissement
2,2394191,2017-11-06,11991,75117,2 Roues Motorisées,Conducteur,Blessé léger,37.0,Masculin,En-Agg,...,1.0,NaN,NaN,"Accident Léger non mortel, En agglomération, E...","48.8858, 2.32163",Paris 17e Arrondissement,75117,"{""coordinates"": [[[[2.303774362, 48.894153779]...","48.8858, 2.32163",Paris 17e Arrondissement


In [126]:
# Rename columns to English-friendly names for our beloved readers

df = df.rename(columns={
    'IdUsager': 'victim_ID',
    'Date': 'accident_date',
    'PV': 'report_number',
    'Mode': 'victim_transport_mode',
    'Catégorie': 'victim_category',
    'Gravité': 'victim_injury_severity',
    'Age': 'victim_age',
    'Genre': 'victim_sex',
    'Milieu': 'environment',
    'Adresse': 'address',
    'Id accident': 'accident_ID',
    'PIM/BD PERIPHERIQUE': 'periphery_info',
    "Tranche d'age": 'victim_age_group',
    'Blessés Légers': 'victim_minor_injuries?',
    'Blessés hospitalisés': 'victim_hospitalized?',
    'Tué': 'victim_deceased?',
    'Résumé': 'report_summary',
    'Nom arrondissement': 'district_name',           
    'Nom arrondissement.1': 'district_name.1',
    'Coordonnées': 'coordinates',
    'Coordonnées.1': 'coordinates.1',
    'Arrondissement': 'district',
    'arronco': 'district_code',
    'Latitude': 'latitude',
    'Longitude': 'longitude'
})

df.head(3)


,victim_ID,accident_date,report_number,district,victim_transport_mode,victim_category,victim_injury_severity,victim_age,victim_sex,environment,...,victim_minor_injuries?,victim_hospitalized?,victim_deceased?,report_summary,coordinates,district_name,district_code,arrondgeo,coordinates.1,district_name.1
0,2389401,2017-04-03,3527,75111,Piéton,Piéton,Blessé léger,62.0,Feminin,En-Agg,...,1.0,NaN,NaN,"Accident Léger non mortel, En agglomération, H...","48.855, 2.36867",Paris 4e Arrondissement,75104,"{""coordinates"": [[[[2.369123881, 48.853166231]...","48.855, 2.36867",Paris 4e Arrondissement
1,2388322,2017-08-28,9113,75108,2 Roues Motorisées,Conducteur,Blessé léger,30.0,Masculin,En-Agg,...,1.0,NaN,NaN,"Accident Léger non mortel, En agglomération, E...","48.8667, 2.3013",Paris 8e Arrondissement,75108,"{""coordinates"": [[[[2.301737288, 48.863496077]...","48.8667, 2.3013",Paris 8e Arrondissement
2,2394191,2017-11-06,11991,75117,2 Roues Motorisées,Conducteur,Blessé léger,37.0,Masculin,En-Agg,...,1.0,NaN,NaN,"Accident Léger non mortel, En agglomération, E...","48.8858, 2.32163",Paris 17e Arrondissement,75117,"{""coordinates"": [[[[2.303774362, 48.894153779]...","48.8858, 2.32163",Paris 17e Arrondissement


Let's create a summary of each column, including data types, missing values, and unique values. This helps us understand the data's structure and identify potential issues.

In [127]:
def column_summary(df):
    summary_data = []
    
    for col_name in df.columns:
        col_dtype = df[col_name].dtype
        num_of_nulls = df[col_name].isnull().sum()
        num_of_non_nulls = df[col_name].notnull().sum()
        num_of_distinct_values = df[col_name].nunique()
        
        summary_data.append({
            'col_name': col_name,
            'col_dtype': col_dtype,
            'num_of_nulls': num_of_nulls,
            'num_of_non_nulls': num_of_non_nulls,
            'num_of_distinct_values': num_of_distinct_values,
        })
    
    summary_df = pd.DataFrame(summary_data)
    return summary_df
summary_df = column_summary(df)
display(summary_df)

,col_name,col_dtype,num_of_nulls,num_of_non_nulls,num_of_distinct_values
0,victim_ID,int64,0,41211,41211
1,accident_date,object,0,41211,2550
2,report_number,int64,0,41211,7105
3,district,int64,0,41211,40
4,victim_transport_mode,object,0,41211,6
5,victim_category,object,0,41211,3
6,victim_injury_severity,object,0,41211,3
7,victim_age,float64,0,41211,103
8,victim_sex,object,0,41211,2
9,environment,object,0,41211,2


This summary suggests that some columns might be duplicates (e.g., DistrictName and DistrictName.1, Coordinates and Coordinates.1). We'll investigate these further.

In [128]:
district_mismatch = df[
    df['district_name'].notna() &
    (df['district_name'] != df['district_name.1'])
]

The check shows no differences, indicating that both `DistrictName` columns are redundant. I then proceed to verify whether the `Coordinates` and `Coordinates.1` columns, as well as the `Longitude` and `Latitude` columns, also contain exactly the same information. This consistency is important to confirm before dropping duplicates. Below is a simple equality check to see if any row has mismatched coordinate data. If it yields an empty result, the columns match perfectly.


In [129]:
coord_mismatch = df[
    df['coordinates'].notna() &
    (df['coordinates'] != df['coordinates.1'])
]

latlong_mismatch = df[
    (df['longitude'].notna()) & 
    (df['latitude'].notna()) &
    (df['coordinates'].notna()) &
    (df.apply(lambda row: f"{row['latitude']}, {row['longitude']}", axis=1) != df['coordinates'])
]
latlong_mismatch

,victim_ID,accident_date,report_number,district,victim_transport_mode,victim_category,victim_injury_severity,victim_age,victim_sex,environment,...,victim_minor_injuries?,victim_hospitalized?,victim_deceased?,report_summary,coordinates,district_name,district_code,arrondgeo,coordinates.1,district_name.1


Both resulting DataFrames (coord_mismatch and latlong_mismatch) are empty, I am confident that `Coordinates`, `Coordinates.1`, `Longitude`, and `Latitude` all store the same location details.


### Arrondissement data quality check

From the summary step, we also notice other arrondissement-related columns, such as `District` and `arronco`, which might differ slightly. Often, `DistrictName` (probably police data) and `arronco` (probably geocoding-based) can conflict near arrondissement boundaries.

To visually confirm the boundary issue, I opened Google Maps at the coordinates of an accident with conflicting district data (48.855, 2.3687). 
![alt text](image-1.png)

I see that the accident occurred right where the 4th and 11th districts meet. The police data labeled it as the 11th, whereas the geocoding approach correctly placed it in the 4th.

**This validates my decision to keep only one reliable district-related column in the final dataset. I choose `arronco` because it is derived from geolocation and typically has fewer missing values, thus more precise.**

Below, I remove columns that store data I no longer need (e.g., duplicates) or less accurate versions. I also convert remaining columns to their adequate data types and save the dataframe to a new CSV file.

In [122]:
columns_to_drop = [
    'district_name',     # Redundant if we keep geocoding-based district_code
    'district_name.1',   # Also redundant
    'coordinates',       # Redundant if we rely on Latitude/Longitude
    'coordinates.1',     # Also redundant
    'district',          # Less reliable than "district_code"
    'Champ13',          # Redundant
    'victim_injury_severity', # Redundant
    'victim_ID', # Not needed for analysis
    'report_number', # Not needed for analysis
]
df = df.drop(columns=columns_to_drop)

# Convert 'Date' column to datetime
df['accident_date'] = pd.to_datetime(df['accident_date'], errors='coerce')

# List of numeric columns
numeric_cols = [
    'victim_age', 'accident_ID', 
    'victim_minor_injuries?', 'victim_hospitalized?', 'victim_deceased?'
]

# Convert numeric columns to nullable integer types
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce').astype('Int64')

# List of string columns
string_cols = [
    'victim_transport_mode', 'victim_category', 
    'victim_sex', 'environment', 'address', 'periphery_info', 
    'victim_age_group', 'report_summary'
]


# Convert string columns to the Pandas string dtype
df[string_cols] = df[string_cols].astype('string')

# Convert `victim_minor_injuries?`, `victim_hospitalized?`, and `victim_deceased?` to booleans
bool_cols = ['victim_minor_injuries?', 'victim_hospitalized?', 'victim_deceased?']
df[bool_cols] = df[bool_cols].map(lambda x: True if x == 1 else False)

# Update `victim_sex` to use `M` or `F`
df['victim_sex'] = df['victim_sex'].map({'Masculin': 'M', 'Feminin': 'F'})

# Save the new df
df.to_csv('../data/accidents_cleaned.csv',index=False, sep=';' )

## Data preparation - Let's look closer at the `Summary` column.

Looking at rows details, we seem to have a enough information for an analysis of Paris traffic accidents. If we want to go deeper, it appears that the `Summary` column often offers a short, free-text narrative about how each accident occurred.

Now, we're embarking on the data parsing phase of our project. This is where we take the raw, messy text summaries of traffic accidents and transform them into structured data that we can analyze. You might think this would be straightforward, but it's surprisingly tricky.

Here's the problem: These summaries are full of incomplete phrases, like in this example: "Minor accident, Non-fatal, In urban area, T-intersection, Daylight, with Normal weather and Normal road surface. 1 Passenger vehicle (PV) traveling on Municipal Road (MR) driven by 1 Male user, 30 years old (Ind) hits 1 Veh". What does "Veh" stand for? Or consider this one: "Minor accident, Non-fatal, In urban area, Not at an intersection, Daylight, with Normal weather and Normal road surface. 1 Bicycle traveling on Municipal Road (MR) ridden by 1 Male user, 19 years old (Ind) hits 1 Pedestrian Male d". What comes after "d"? We can guess, but we need to be certain. It's not just about filling in the blanks; the structure of the summaries varies, and they use a lot of specialized abbreviations, like "VMA" for the maximum speed limit or "EDP-m" for a motorized personal mobility device.

We're not going to use fancy generative AI models for this task, though. Why? Because we need absolute accuracy and control. These AI models are great at generating text that looks right, but they can make mistakes, and we can't afford that when dealing with data that we'll be using for analysis. Plus, we need to understand exactly why a correction was made, and these models are like black boxes – it's hard to know what's going on inside. We'll use a more transparent and reliable approach, combining carefully crafted rules with some clever techniques to handle these tricky text snippets.